# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = 'data/amazon_products.csv'
path_B = 'data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2976
Number of tuples in B: 4847
Number of tuples in A X B (i.e the cartesian product): 14424672


In [6]:
A.head()

,id,product title,brand,model,operating system,combo
0,a0,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,nh.q28aa.001,windows 10,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,acer,e5-576g-5762,windows 10,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...
2,a2,asus vivobook f510ua fhd laptop intel core i5-8250u 8gb ram 1tb hdd usb-c nanoedge display finge...,asus,f510ua-ah51,windows 10,asus vivobook f510ua fhd laptop intel core i5-8250u 8gb ram 1tb hdd usb-c nanoedge display finge...
3,a3,asus c302ca-dhm4 chromebook flip 12.5-inch touchscreen convertible chromebook intel core m3 4gb ...,asus,c302ca-dhm4,chrome,asus c302ca-dhm4 chromebook flip 12.5-inch touchscreen convertible chromebook intel core m3 4gb ...
4,a4,asus chromebook c202sa-ys02 11.6 ruggedized and water resistant design with 180 degree (intel ce...,asus,c202sa-ys02,chrome,asus chromebook c202sa-ys02 11.6 ruggedized and water resistant design with 180 degree (intel ce...


In [7]:
B.head()

,id,product title,brand,model,operating system,combo
0,w0,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10,iview,NaN,microsoft windows,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10 iview micr...
1,w1,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red,dell,i3168-3270red,windows 10,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,w2,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...,iview,max2-bk,windows 10,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...
3,w3,direkt-tek 11.6 convertible touchscreen laptop windows 10 office 365 personal 1-year subscriptio...,direkt-tek,dtlapy116-2-sl,microsoft windows,direkt-tek 11.6 convertible touchscreen laptop windows 10 office 365 personal 1-year subscriptio...
4,w4,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...,dell,i3168-0027red,windows 10,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [9]:
# Blocking plan

# A, B -- attribute equiv. blocker [model] --------------------|---> candidate set

In [10]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Use block_tables to apply blocking over two input tables.
C1 = ob.block_tables(A,B, 'model', 'model', 
                    l_output_attrs=['id','model','combo'], 
                    r_output_attrs=['id','model','combo'],
                    overlap_size=8,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


2635

In [11]:
C1.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a229,w1,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a1922,w7,i5578-5902gry,dell i5578-5902gry inspiron pro 15.6 fhd with touch laptop computer (core i5-7200u 8gb ddr4 256g...,i5578-3093gry,dell - inspiron 15 5000 2-in-1 15.6 touch-screen laptop - intel core i3 - 4gb memory - 500gb 540...
2,2,a229,w12,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3271blu,dell - inspiron 11 3000 2-in-1 blue 11.6-inch hd intel pentium processor n3710 4gb 1600mhz ddr3l...
3,3,a1561,w15,i5579-7050gry,dell inspiron 13 i5379-7302gry-pus 2 in 1 pc i5579-7050gry windows 10,i5578-2550gry,dell - inspiron 15 5000 2-in-1 15.6-inch fhd intel core i7-7500u 8gb 2400mhz ddr4 1tb 5400 rpm h...
4,4,a1922,w15,i5578-5902gry,dell i5578-5902gry inspiron pro 15.6 fhd with touch laptop computer (core i5-7200u 8gb ddr4 256g...,i5578-2550gry,dell - inspiron 15 5000 2-in-1 15.6-inch fhd intel core i7-7500u 8gb 2400mhz ddr4 1tb 5400 rpm h...


## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [12]:
# Debug blocker output
dbg = em.debug_blocker(C1, A, B, output_size=10, 
                       attr_corres=[
                           ('product title','product title'),
                           ('brand', 'brand'), 
                           ('model','model'),
                           ('combo','combo')],
                      verbose=True)
#### Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,ltable_id,rtable_id,ltable_product title,ltable_brand,ltable_model,ltable_combo,rtable_product title,rtable_brand,rtable_model,rtable_combo
0,0,a1116,w434,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,michaelelectronics2,NaN,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,michaelelectronics2,hp-15-02395-fhd-me28,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
1,1,a1341,w2999,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,michaelelectronics2,NaN,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,michaelelectronics2,asu-14-07054-fhd-me4,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...
2,2,a28,w2192,asus fx503vm 15.6” fhd powerful gaming laptop intel core i7-7700hq quad-core 2.8ghz (turbo up to...,asus,fx503vm-eh73,asus fx503vm 15.6” fhd powerful gaming laptop intel core i7-7700hq quad-core 2.8ghz (turbo up to...,asus fx503vm 15.6” fhd intel core i7-7700hq quad-core 2.8ghz (turbo up to 3.8ghz) 3gb gtx 1060 1...,asus,90nr0gp1-m00060,asus fx503vm 15.6” fhd intel core i7-7700hq quad-core 2.8ghz (turbo up to 3.8ghz) 3gb gtx 1060 1...
3,3,a2110,w734,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...,lg,NaN,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...,lg,15z970-a.aas7u1,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...
4,4,a1161,w376,lenovo yoga y720 13 premium thin light convertible 2 in 1 laptop pc (intel 8th gen i7 quad core ...,michaelelectronics2,NaN,lenovo yoga y720 13 premium thin light convertible 2 in 1 laptop pc (intel 8th gen i7 quad core ...,lenovo yoga y720 13 premium thin light convertible 2 in 1 laptop pc (intel 8th gen i7 quad core ...,michaelelectronics2,len-13-01235-fhd-me4,lenovo yoga y720 13 premium thin light convertible 2 in 1 laptop pc (intel 8th gen i7 quad core ...


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [13]:
# Updated blocking sequence
# A, B ------ overlap blocker [combo] --------> C1--
#                                                   |----> C
# A, B ------ overlap blocker [model] --------> C2--

In [14]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'combo' attribute 
C2 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['id','combo'], 
                    r_output_attrs=['id','combo'],
                    overlap_size=80,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


5925

In [15]:
# Display first two rows from C2
C2.head()

,_id,ltable_id,rtable_id,ltable_combo,rtable_combo
0,0,a322,w32,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...
1,1,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
2,2,a621,w41,asus vivobook flip 14 tp401ca-dhm6t 14” thin and lightweight 2-in-1 hd touchscreen laptop intel ...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
3,3,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
4,4,a432,w72,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display windo...,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display dell ...


We add another blocker.

In [16]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C1,C2])
len(C)

8422

In [17]:
C.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a0,w1389,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1taa.001,refurbished acer 17.3 intel core i7 2.8 ghz 16 gb ram 1 tb hdd + 256 gb ssd windows 10 home acer...
1,1,a0,w1396,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1aaa.001,manufacturer refurbished acer predator g9-793-79pe 17.3 intel core i7-6700hq 2.6 ghz 16gb ram 1t...
2,2,a0,w1504,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1faa.001,refurbished acer 17.3 intel core i7 2.9 ghz 32 gb ram 1 tb hdd + 512 gb ssd windows 10 home acer...
3,3,a0,w1516,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1eaa.001,refurbished acer 17.3 intel core i7 2.9 ghz 32 gb ram 1 tb hdd + 512 gb ssd windows 10 home acer...
4,4,a0,w1661,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q13aa.001,refurbished acer 17.3 intel core i7 2.7 ghz 32 gb ram 1 tb hdd windows 10 home|gx-791-758v acer ...


We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [18]:
# Debug again
dbg = em.debug_blocker(C, A, B, output_size=10)
dbg.head()

,_id,ltable_id,rtable_id,ltable_product title,ltable_brand,ltable_model,ltable_operating system,ltable_combo,rtable_product title,rtable_brand,rtable_model,rtable_operating system,rtable_combo
0,0,a1031,w4480,acer laptop 15.6 intel core i5 3.40ghz 8gb ram 256gb ssd windows 10 (certified refurbished),acer,a515-51-596k,windows 10 home,acer laptop 15.6 intel core i5 3.40ghz 8gb ram 256gb ssd windows 10 (certified refurbished) acer...,refurbished acer laptop 15.6 intel core i5 3.40ghz 8gb ram 256gb ssd windows 10,acer,nx.gtpaa.002,windows 10,refurbished acer laptop 15.6 intel core i5 3.40ghz 8gb ram 256gb ssd windows 10 acer nx.gtpaa.00...
1,1,a1257,w4411,hp elitebook 850 g3 15.6' fhd laptop (intel i7-6500u 16 gb memory 256gb ssd windows 10 pro 64) w...,hp,t8tjg,NaN,hp elitebook 850 g3 15.6' fhd laptop (intel i7-6500u 16 gb memory 256gb ssd windows 10 pro 64) w...,refurbished hp elitebook 850 g3 15.6' fhd laptop (intel i7-6500u 16 gb memory 1tb ssd windows 10...,hp,NaN,NaN,refurbished hp elitebook 850 g3 15.6' fhd laptop (intel i7-6500u 16 gb memory 1tb ssd windows 10...
2,2,a317,w687,2018 dell - inspiron i3567-3657blk-pus 15.6 touch-screen laptop - intel core i3-7100u - 6gb memo...,dell,i3567-3657blk-pus,windows 10,2018 dell - inspiron i3567-3657blk-pus 15.6 touch-screen laptop - intel core i3-7100u - 6gb memo...,dell - inspiron 15.6 touch-screen laptop - intel core i5 - 8gb memory - 1tb hard drive - black,dell,NaN,windows 10,dell - inspiron 15.6 touch-screen laptop - intel core i5 - 8gb memory - 1tb hard drive - black d...
3,3,a1164,w802,dell flagship inspiron 2-in-1 13.3 touch-screen laptop - intel core i5 -7200u - 8gb memory - 256...,dell,t8tjg,windows 10,dell flagship inspiron 2-in-1 13.3 touch-screen laptop - intel core i5 -7200u - 8gb memory - 256...,dell latitude 2-in-1 13.3 touch-screen laptop - intel core i5 - 8gb memory - 256gb solid state d...,dell,lati7389628gv,windows 10,dell latitude 2-in-1 13.3 touch-screen laptop - intel core i5 - 8gb memory - 256gb solid state d...
4,4,a20,w783,microsoft surface book 2 (intel core i7 16gb ram 512gb) - 15,microsoft,fux-00001,windows 10 pro,microsoft surface book 2 (intel core i7 16gb ram 512gb) - 15 microsoft fux-00001 windows 10 pro,microsoft surface book 2 (intel core i7 16gb ram 1tb) - 15,microsoft,fvh-00001,windows 10 pro creators update,microsoft surface book 2 (intel core i7 16gb ram 1tb) - 15 microsoft fvh-00001 windows 10 pro cr...


We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [19]:
# Sample  candidate set
# S = em.sample_table(C, 500)
# em.to_csv_metadata(S, 'data/new.csv')
# S.head()

For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [20]:
# # Load the pre-labeled data
path_G = 'data/labeled.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

300

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [21]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [22]:
G.head()


,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo,match
0,27,a1011,w685,t8tjg,2018 flagship lenovo thinkpad 11e (4h generation) 11.6 notebook intel i3-7100u 128gb m.2 ssd 4gb...,t8tjg,dell chromebook 11 3189 - 11.6 - celeron n3060 - 4 gb ram - 64 gb ssd dell t8tjg chrome os,0
1,63,a1044,w915,80xb000bus,lenovo flex 5 15.6 full hd touch multimode 2-in-1 notebook computer intel core i7-7500u 16gb ram...,80ys000bus,topseller n23 n3060 1.60g 4gb 32gb 11.6in hd display chrome lenovo 80ys000bus chrome os,0
2,99,a1047,w1550,n850hp6,prostar clevo gaming laptop n850hp6 15.6” fhd ips (1920x1080) led matte type display intel core ...,n850hk1,prostar clevo gaming laptop n850hk1 15.6” fhd ips (1920x1080) matte type display intel core i7-7...,1
3,100,a1047,w1553,n850hp6,prostar clevo gaming laptop n850hp6 15.6” fhd ips (1920x1080) led matte type display intel core ...,n850hp6,prostar clevo gaming laptop n850hp6 15.6” fhd ips (1920x1080) led matte type display intel core ...,1
4,134,a1047,w1623,n850hp6,prostar clevo gaming laptop n850hp6 15.6” fhd ips (1920x1080) led matte type display intel core ...,p650hp6-g,prostar clevo p650hp6-g vr ready gaming 15.6 fhd/ips/matte display with g-sync intel core i7-770...,1


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [23]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [24]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
feature_table = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)

In [25]:
def generateContainsValueFeature(value, attribute='combo'):
    def containsValueFeature(a,b):
        return int((value.lower() in a[attribute].lower()) == (value.lower() in b[attribute].lower()))
    return containsValueFeature, value

In [26]:
brands = ['hp','dell','lenovo','prostar','acer','samsung','apple','asus','panasonic']
models = ['xps','carbon','x1','yoga','envy','thinkpad','lattitude','inspiron','elitebook','clevo','spectre','macbook','pavilion','ideapad','legion']
sizes = ['13','15','17']
operating_systems = ['chrome','windows','mac']
cpus = ['i3','i5','i7','celeron','pentium']
keywords = brands + models + sizes + operating_systems + cpus
new_features = [generateContainsValueFeature(value) for value in keywords]

for feature in new_features:
    em.add_blackbox_feature(feature_table, feature[1], feature[0])

In [27]:
# List the names of the features generated
feature_table['feature_name']

0                                            id_id_lev_dist
1                                             id_id_lev_sim
2                                                 id_id_jar
3                                                 id_id_jwn
4                                                 id_id_exm
5                                     id_id_jac_qgm_3_qgm_3
6               product_title_product_title_jac_qgm_3_qgm_3
7           product_title_product_title_cos_dlm_dc0_dlm_dc0
8                               brand_brand_jac_qgm_3_qgm_3
9                           brand_brand_cos_dlm_dc0_dlm_dc0
10                          brand_brand_jac_dlm_dc0_dlm_dc0
11                                          brand_brand_mel
12                                     brand_brand_lev_dist
13                                      brand_brand_lev_sim
14                                          brand_brand_nmw
15                                           brand_brand_sw
16                              model_mo

### Converting the development set to feature vectors

In [28]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)
H.fillna(0, inplace=True)
H.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,product_title_product_title_jac_qgm_3_qgm_3,...,17,chrome,windows,mac,i3,i5,i7,celeron,pentium,match
282,7036,a888,w3248,4,0.2,0.483333,0.483333,0,0.083333,0.504237,...,1,1,1,1,1,1,1,1,1,0
44,1158,a1347,w3193,5,0.0,0.433333,0.433333,0,0.000000,0.607656,...,1,1,1,1,1,1,1,1,1,1
272,6861,a816,w3647,5,0.0,0.483333,0.483333,0,0.000000,0.266667,...,1,1,1,1,1,0,1,1,1,1
189,4762,a2426,w3223,4,0.2,0.600000,0.600000,0,0.000000,0.515152,...,1,1,1,1,1,0,0,1,1,1
152,3872,a2104,w3121,4,0.2,0.466667,0.466667,0,0.000000,0.835106,...,1,1,1,1,1,0,0,1,1,1


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use the 'precision' metric to select the best matcher.

In [29]:
# Select the best ML matcher using CV
result = em.select_matcher(
        matchers=[dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.947863,0.947527,0.945986
1,RF,0.950853,0.952927,0.951098
2,SVM,0.795514,0.928602,0.853310
3,LinReg,0.914599,0.938283,0.925133
4,LogReg,0.922299,0.972282,0.945743
5,NaiveBayes,0.881740,0.958594,0.916106


### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [30]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [ ]:
# Debug rf using GUI
em.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        target_attr='match')

##  Evaluating the matching output

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [32]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='match', show_progress=False)
H.fillna(0, inplace=True)

In [33]:
# Select the best ML matcher using CV
result = em.select_matcher(
        [dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.947863,0.947527,0.945986
1,RF,0.950853,0.952927,0.951098
2,SVM,0.795514,0.928602,0.853310
3,LinReg,0.914599,0.938283,0.925133
4,LogReg,0.922299,0.972282,0.945743
5,NaiveBayes,0.881740,0.958594,0.916106


Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [34]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(
        J, 
        feature_table=feature_table,
        attrs_after='match', 
        show_progress=False)
L.fillna(0, inplace=True)
L.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,product_title_product_title_jac_qgm_3_qgm_3,...,17,chrome,windows,mac,i3,i5,i7,celeron,pentium,match
208,5189,a2500,w2256,4,0.2,0.600000,0.600000,0,0.000000,0.010753,...,0,1,0,1,1,0,1,1,1,0
188,4747,a2426,w3192,5,0.0,0.466667,0.466667,0,0.000000,0.442688,...,1,1,1,1,1,0,0,1,1,0
12,437,a1138,w3417,5,0.0,0.466667,0.466667,0,0.000000,0.583333,...,1,1,1,1,1,1,1,1,1,1
221,5444,a2671,w2981,3,0.4,0.600000,0.600000,0,0.076923,0.363248,...,1,1,1,1,1,1,1,1,1,0
239,5908,a2944,w1770,5,0.0,0.000000,0.000000,0,0.000000,0.000000,...,0,0,1,1,1,0,1,0,1,0


### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [35]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [36]:
# Predict on L 
predictions = rf.predict(
        table=L, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
        append=True, 
        target_attr='predicted', 
        inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [37]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 92.86% (52/56)
Recall : 91.23% (52/57)
F1 : 92.04%
False positives : 4 (out of 56 positive predictions)
False negatives : 5 (out of 34 negative predictions)
